## Agents in LlamaIndex

> Unit 1: An agent is a system that leverages an AI model to interact with its environment to achieve a user-defined objective. It combines reasoning, planning and action execution (often via external tools) to fulfil tasks.

`LlamaIndex` supports 3 main types of reasoning agents:
- Function Calling Agents: These work with AI models that can call specific functions.
- ReAct Agents: These work with any AI model that does chat or text endponit and deal with complex reasoning tasks.
- Advanced Custom Agents: use more complex methods to deal with complex workflows and tasks.

### Basic Agents

In [1]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow

# define a simple tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b


from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(model='gpt-4o-mini')

# ccreate agent
agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools."
)


In [2]:
handler = agent.run('What is (2+2) * 2 ?')

from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print('')
        print(f'Called Tool: {ev.tool_name} {ev.tool_kwargs} => {ev.tool_output}')
    elif isinstance(ev, AgentStream):   # shows thought process/reasoning steps
        print(ev.delta, end="", flush=True)

resp = await handler
resp

        


Called Tool: add {'a': 2, 'b': 2} => 4

Called Tool: multiply {'a': 4, 'b': 2} => 8
The result of (2 + 2) * 2 is 8.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The result of (2 + 2) * 2 is 8.')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 2, 'b': 2}, tool_id='call_GfVMyUrONBYNM4FefcCtglhH', tool_output=ToolOutput(content='4', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False), ToolCallResult(tool_name='multiply', tool_kwargs={'a': 4, 'b': 2}, tool_id='call_3TZWXlu7U4yQSlsroiWMoGKA', tool_output=ToolOutput(content='8', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 4, 'b': 2}}, raw_output=8, is_error=False), return_direct=False)], raw={'id': 'chatcmpl-BRh3KiAEG8vdimbU0jSwCFe4CgASr', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1745940050, 'model': 'gpt-4o-mini-2024-07

#### Agents are stateless by default
So we need to add memory to the agent, and this is done through a `Context` object. This comes handy, for instance, in a chatbot app where the agent needs to remember previous interactions (or a task manager tracking progress over time)

Note: didn't quite work in the example below :(

In [3]:
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run('My name is bob', ctx=ctx)
print(f'Response: {response}')
response = await agent.run('What is my name again?', ctx=ctx)
print(f'Response: {response}')

Response: Hello, Bob! How can I assist you today?
Response: Your name is Bob.


### Creating RAG Agents with QueryEngineTools

We can pass various tools to Alfred to help him answer questions. However, instead of answering the question on top of documents automatically, Alfred can decide to use any other tool or flow to answer the question.

In [4]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.tools import QueryEngineTool
from dotenv import load_dotenv

# Create vector store
db = chromadb.PersistentClient('./alfred_chroma_db')
chroma_collection = db.get_or_create_collection('alfred')
vector_store = ChromaVectorStore(chroma_collection)

# Create a query engine
embed_model = HuggingFaceEmbedding('BAAI/bge-small-en-v1.5')


load_dotenv()
llm = OpenAI(model='gpt-4o-mini')
index = VectorStoreIndex.from_vector_store(
    vector_store, 
    embed_model=embed_model)

query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name='personas',
    description='Query descriptions for various personas',
    return_direct=False
)

# Create a RAG Agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions.",
)

In [5]:
handler = query_engine_agent.run(
    "Search the database for 'traveler' and return some persona descriptions."
)

async for ev in handler.stream_events():
    if isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)
    elif isinstance(ev, ToolCallResult):
        print(f"\nCalled Tool: {ev.tool_name} {ev.tool_kwargs} => {ev.tool_output}")

resp = await handler
print(f"\nResponse: {resp}")


Called Tool: personas {'input': 'traveler'} => The information provided does not relate to a traveler. It focuses on an environmental historian or urban planner with an emphasis on ecological conservation and sustainability.

Called Tool: personas {'input': 'travel'} => Travel can encompass various aspects, including exploring new places, understanding different cultures, and experiencing nature. For someone focused on ecological conservation and sustainability, travel might involve visiting natural reserves, studying urban planning in different regions, or participating in environmental initiatives. This type of travel can provide valuable insights into sustainable practices and the impact of urban development on the environment.
It seems that the database does not have specific persona descriptions for a "traveler." Instead, it provides insights related to travel in the context of ecological conservation and sustainability. This perspective emphasizes exploring new places, understan

### Creating multi-agent systems
Passing multiple agents to the `AgentWorkflow` class

In [6]:
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    ReActAgent
)

# define some tools:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = ReActAgent(
    name='calculator',
    description='Performs basic arithmetic operations',
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools = [add, subtract],
    llm=llm
)

query_agent = ReActAgent(
    name='info_lookup',
    description='Looks up information about XYZ',
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools = [query_engine_tool],
    llm=llm
)

agent = AgentWorkflow(
    agents= [calculator_agent, query_agent],
    root_agent='calculator'
)

handler = agent.run(user_msg='can you add 5 and 3?')

In [7]:
async for ev in handler.stream_events():
    if isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)
    elif isinstance(ev, ToolCallResult):
        print(f'\nCalled Tool: {ev.tool_name}, {ev.tool_kwargs} => {ev.tool_output}')

resp = await handler
print(f'\nResponse: {resp}')
        

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 5, "b": 3}
Called Tool: add, {'a': 5, 'b': 3} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The sum of 5 and 3 is 8.
Response: The sum of 5 and 3 is 8.
